In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile


IMAGE_SIZE = [150, 150]

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 20,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    shear_range = 0.1,
                                    zoom_range = 0.1,
                                    horizontal_flip = True
                                   
                                   )

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory('processed_images/train',
                                                    batch_size = 32,
                                                    class_mode = 'categorical', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( 'processed_images/test',
                                                          batch_size  = 32,
                                                          class_mode  = 'categorical', 
                                                          target_size = (150, 150),
                                                          shuffle = False 
                                                    )



Found 491 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


In [3]:
model = Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(150, 150, 3)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Flatten())  

model.add(Dropout(0.4))  

model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))


model.add(Dense(2, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.0001)))  



model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 73, 73, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 73, 73, 64)        256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 73, 73, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)       

In [ ]:
import time
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


start = time.time()

model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,  
    callbacks=[reduce_lr, early_stopping]
)

end = time.time()
elapsed = end - start
print("Total Time:", elapsed)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.4789 - loss: 0.7262 - val_accuracy: 0.5041 - val_loss: 0.7095 - learning_rate: 0.0010
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 726ms/step - accuracy: 0.5830 - loss: 0.6980 - val_accuracy: 0.5041 - val_loss: 0.7121 - learning_rate: 0.0010
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 827ms/step - accuracy: 0.6372 - loss: 0.6328 - val_accuracy: 0.5041 - val_loss: 0.7404 - learning_rate: 0.0010
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 801ms/step - accuracy: 0.6864 - loss: 0.5898 - val_accuracy: 0.5041 - val_loss: 0.7941 - learning_rate: 0.0010
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 826ms/step - accuracy: 0.7029 - loss: 0.6034 - val_accuracy: 0.5041 - val_loss: 0.8186 - learning_rate: 0.0010
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 733ms/step - accuracy: 0.7269 - loss: 0.5361 - val_accuracy: 0.5041 - val_loss: 0.7917 - learning_rate: 0.0010
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 697ms/step - accuracy: 0.7636 - loss: 0.532

In [16]:
loss, accuracy = model.evaluate(validation_generator)
print("Accuracy score:", accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 686ms/step - accuracy: 0.9097 - loss: 0.2013
Accuracy score: 0.9173553586006165


In [18]:
# Save the model in HDF5 format
model.save('my_model.h5')


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(validation_generator)  
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

target_names = ['Normal', 'Pneumonia']
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 693ms/step
Confusion Matrix
[[54  7]
 [ 3 57]]
Classification Report
              precision    recall  f1-score   support

      Normal       0.95      0.89      0.92        61
   Pneumonia       0.89      0.95      0.92        60

    accuracy                           0.92       121
   macro avg       0.92      0.92      0.92       121
weighted avg       0.92      0.92      0.92       121

